In [1]:
import torch
import os
import numpy as np
from dotenv import load_dotenv
from pinecone import Pinecone, PodSpec
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import time
from huggingface_hub import InferenceClient

In [2]:
cache_dir="cache_dir"
load_dotenv('./.env')

True

#### Leitura do arquivo e criação do splitter'

In [3]:
with open('docs/nat.txt') as f:
    clt = f.read()

# Criação de um objeto RecursiveCharacterTextSplitter para dividir o texto em pedaços
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=10,
    length_function=len
)

In [4]:
chunks = text_splitter.create_documents([clt])
text_chunks = [doc.page_content for doc in chunks]

In [5]:
# Inicialize o modelo de embedding
model = SentenceTransformer('all-MiniLM-L6-v2')

# Isso retornará uma lista de vetores
embeddings = model.encode(text_chunks, convert_to_tensor=False)  

# Adiciona zeros para estender cada vetor até a dimensão 1536
extended_embeddings = np.array([np.pad(emb, (0, 1536 - len(emb)), 'constant') for emb in embeddings])

In [6]:
pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))
indexes = pc.list_indexes()
index_name = 'natgpt'

In [7]:
for i in indexes:
    pc.delete_index(i['name'])
    print('Index encontrada e apagada: ' + i['name'])

if index_name not in pc.list_indexes():
    pc.create_index(index_name, dimension=1536, metric='cosine', spec=PodSpec(environment=os.environ.get('PINECONE_ENV')))
    print('Index '+index_name+' criado')

Index encontrada e apagada: natgpt
Index natgpt criado


In [8]:
# Conecta na index criada
pc_index = pc.Index(index_name)

In [9]:
documents_to_insert = []
for i, embedding in enumerate(extended_embeddings):
    doc_id = f"{i}"
    documents_to_insert.append({"id": doc_id, "values": embedding.tolist()})

In [10]:
# Insere os documentos na index do Pinecone
pc_index.upsert(vectors=documents_to_insert)

# Aguarda 20 segundos para dar tempo de atualizar a Index no Pinecone
time.sleep(20)

### Busca por similaridade

In [11]:
# Inicialização do modelo para perguntas e respostas
llm = InferenceClient(model="mistralai/Mistral-7B-Instruct-v0.1")

In [12]:
def perguntar(prompt, top_k=3, debug=False):
    """
    Recebe um prompt do usuário para fazer uma busca semântica, encontra os resultados
    correspondentes no documento original e passa o resultado como contexto ao LLM

    Parâmetros:
    - prompt (str): Prompt do usuário. Também é usado para a busca semântica
    - top_k (int): Quantidade de resultados mais semalhantes
    - debug (bool): Retorna (ou não) o contexto passado ao LLM

    Retorna:
    - Tipo do retorno: A resposta do LLM com base na pergunta e contexto
    """
    
    # Gerando o vetor de consulta
    query_vector = model.encode(prompt, convert_to_tensor=False)
    
    # Adiciona zeros para estender cada vetor até a dimensão 1536
    padding_length = 1536 - len(query_vector)
    padded_vector = np.pad(query_vector, (0, padding_length), 'constant')
    
    query_vector_list = padded_vector.tolist()
    query_result = pc_index.query(vector=query_vector_list, top_k=top_k)
    
    contexto = []
    for index, match in enumerate(query_result.matches):
        contexto.append(text_chunks[int(match.id)])

    input_text = f"[INST]Responda em Português do Brasil exatamente o que foi perguntado. Não invente nenhuma informação: {prompt}.\n\nContexto: {contexto}[/INST]"

    response = llm.text_generation(
        input_text,
        temperature=0.2, 
        max_new_tokens=500,
        top_k=30,
        top_p=0.9,
        repetition_penalty=1.0
    )

    if(debug == True):
        response = response + "\n\n\nDebug\n---------------------------------\n" + str(contexto)

    return response

### Perguntas com base no contexto passado

In [13]:
print(
    perguntar(
        prompt='Quem são os coordenadores do nat?', top_k=4, debug=False
    )
)

 Os coordenadores do NAT são Marcela Cristina Ozório e Bernardo Fiterman Albano.


In [14]:
print(
    perguntar(
        prompt='Qual o site do nat?', top_k=4, debug=False
    )
)

 O site do NAT é: <https://nat.mpac.mp.br>.


In [15]:
print(
    perguntar(
        prompt='Quais os principais setores do NAT?', top_k=4, debug=False
    )
)

 O NAT (Núcleo de Apoio Técnico) é um órgão administrativo auxiliar de apoio técnico especializado do Ministério Público do Estado do Acre. Os principais setores do NAT são: Coordenação de Desenvolvimento de Sistemas, Observatório Criminal, LAB (Laboratório de Tecnologia contra a Lavagem de Dinheiro), Observatório de Políticas Públicas e Técinco Científico.


In [16]:
print(
    perguntar(
        prompt='Me da um resumo do NAT?', top_k=4, debug=False
    )
)

 O NAT (Núcleo de Apoio Técnico) é um órgão do MPAC (Ministério Público do Acre) criado em 2012. Ele tem como função atender a demandas de processos complexos e intersetoriais, onde todos os setores atuam de forma integrada. O NAT tem como principais setores a Coordenação de Desenvolvimento de Sistemas, Observatório Criminal, LAB (Laboratório de Tecnologia contra a Lavagem de Dinheiro), Observatório de Políticas Públicas e Técinco Científico. O site do NAT é https://nat.mpac.mp.br.


In [17]:
print(
    perguntar(
        prompt='Me da um resumo do Observatório de Políticas Públicas', top_k=4, debug=False
    )
)

 O Observatório de Políticas Públicas (OPP) é um projeto do Núcleo de Apoio Técnico (NAT), órgão do MPAC (Ministério Público do Acre), criado em 2012. O OPP tem como objetivo monitorar os orçamentos públicos e a aplicação dos recursos, verificando a consonância entre o que foi planejado com o executado e se o resultado almejado foi alcançado e promoveu a transformação social que era buscada. O OPP usará instrumentos formais, como o Plano Plurianual, Lei de Diretrizes Orçamentárias e Lei Orçamentária Anual, para produzir e disponibilizar dados, informações e conhecimentos aos membros do MPAC. O OPP abrangerá as áreas da saúde, educação, meio ambiente, segurança pública e políticas de assistência social.


In [18]:
print(
    perguntar(
        prompt='Qual o site do OPP', top_k=4, debug=False
    )
)

 O site do OPP (Observatório De Políticas Públicas) é: <https://nat.mpac.mp.br/posts/category/observatorio-politicas-publicas/>.


In [19]:
print(
    perguntar(
        prompt='O que é o Retina?', top_k=4, debug=False
    )
)

 O Retina é uma ferramenta gerenciadora de conteúdo, desenvolvida a partir do armazenamento de dados/informações relacionados às organizações criminosas e a seus integrantes. A ferramenta possibilita aos usuários (observadores) o cadastro de fatos formais relacionados a atuaação de facções/ORCRIM e a posterior consulta por meio de filtros relacionados aos integrantes (observados), assim como a geração de análises de correlação e de vínculo entre pessoas, entre fatos e entre pessoas e fatos.


In [20]:
# Pergunta com resposta totalmente fora do contexto
print(
    perguntar(
        prompt='O que são redes neurais artificiais', top_k=4, debug=False
    )
)

 Redes neurais artificiais são sistemas computacionais que são inspirados no funcionamento do cérebro humano. São compostos por uma grande quantidade de neurônios que são interconectados por meio de sinapses, e que trabalham juntos para processar e analisar informação. Redes neurais artificiais são utilizadas em diversos campos, como a inteligência artificial, o reconhecimento de padrões, a previsão de comportamento e a resolução de problemas.
